In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 20 15:17:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
## Unzip dataset (Blender_new)
!mkdir /content/FDA_Drone_Data
!unzip /content/drive/MyDrive/FDA.zip -d /content/FDA_Drone_Data
!rm -rf /content/FDA_Drone_Data/__MACOSX

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_000577.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_001669.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_006106.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_006660.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_000211.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_004077.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_003718.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_002406.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_005369.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_004063.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_002412.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_006674.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_000205.png  
  inflating: /content/FDA_Drone_Data/FDA/labels_1/image_000563.png  
  infla

In [4]:
from pycocotools.coco import COCO
import os
import os.path as osp
import sys
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import cv2
import numpy as np
from pycocotools.coco import COCO
from google.colab.patches import cv2_imshow
from torch.utils.data import DataLoader


class COCOAnnotationTransform(object):
    def __init__(self):
        self.label_map = COCO_LABEL_MAP

    def __call__(self, target, width, height):
        scale = np.array([width, height, width, height])
        res = []
        for obj in target:
            if 'bbox' in obj:
                bbox = obj['bbox']
                label_idx = self.label_map[obj['category_id']] - 1
                final_box = list(np.array([bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]])/scale)
                final_box.append(label_idx)
                res += [final_box]  # [xmin, ymin, xmax, ymax, label_idx]
            else:
                print("No bbox found for object ", obj)

        return res  # [[xmin, ymin, xmax, ymax, label_idx], ... ]


class COCODetection(data.Dataset):
    def __init__(self, image_path, info_file, transform=None,
                 target_transform=None, has_gt=True):
        self.root = image_path
        self.coco = COCO(info_file)
        self.ids = list(self.coco.imgToAnns.keys())  # 标签数目 小于样本数目，说明有的图像没有标签

        if len(self.ids) == 0 or not has_gt:  # 如果没有标签或者不需要GT，则直接使用image
            self.ids = list(self.coco.imgs.keys())
        self.transform = transform
        self.target_transform = target_transform

        self.has_gt = has_gt

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        im, gt, Mask, h, w, num_crowds, name, Cat = self.pull_item(index)
        return im, (gt, Mask, num_crowds), name, Cat

    def pull_item(self, index):
        img_id = self.ids[index]
        if self.has_gt:
            ann_ids = self.coco.getAnnIds(imgIds=img_id) 
            target = self.coco.loadAnns(ann_ids)
        else:
            target = []
        crowd = [x for x in target if ('iscrowd' in x and x['iscrowd'])] ## iscrowd=0: single target.   iscrowd=1: a group of target(overlapping)
        target = [x for x in target if not ('iscrowd' in x and x['iscrowd'])]
        num_crowds = len(crowd)

        # This is so we ensure that all crowd annotations are at the end of the array
        target += crowd
        file_name = self.coco.loadImgs(img_id)[0]['file_name']
        path = osp.join(self.root, file_name)
        img = cv2.imread(path)
        height, width, _ = img.shape
        if len(target) > 0: # 这样图像中有不是crowd的目标
            Cat = []
            Mask = self.coco.annToMask(target[0]).reshape(-1)
            for obj in target:
              mask = self.coco.annToMask(obj).reshape(-1)
              cat = obj['category_id']

              if cat not in Cat:
                Cat.append(cat)
                Mask = np.vstack((Mask, mask))
              else:
                Mask[-1] = Mask[-1] + mask
              
            Mask = Mask.reshape(-1, height, width)
            Mask = Mask[1:]
        if self.target_transform is not None and len(target) > 0:
            target = self.target_transform(target, width, height)
        return torch.from_numpy(img).permute(2, 0, 1), target, Mask, height, width, num_crowds, file_name, Cat


In [5]:
## Write labeled images
%cd /content/FDA_Drone_Data/FDA
!mkdir ./Label_fromJSON

image = '/content/FDA_Drone_Data/FDA/images'
info_json =  '/content/FDA_Drone_Data/FDA/simulated_corrosion_multi_class_aligned_morph_1.json'

if __name__=='__main__':
    dataset = COCODetection(image, info_json)
    loader = DataLoader(dataset)
    count = 0
    for img, label, name, Cat in loader:
        img = np.uint8(img.squeeze().numpy().transpose(1, 2, 0))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        Name = name[0].replace('image', 'label')
        gt, masks, num_crowds = label
        masks = masks.squeeze(0) ## .squeeze(A): Delete A_th dimension. Now we left batch * H * w, each batch represent a category labeled by 'pixel==1'
        label_img = np.zeros_like(gray)
        Cat_np = []
        for i in Cat:
          Cat_np.append(i.numpy().reshape(-1))
        for m in range(masks.size(0)):
          mask = masks[m].numpy()
          label_img[np.where(mask==1)] = Cat_np[m][0]
        cv2.imwrite(osp.join('/content/FDA_Drone_Data/FDA/Label_fromJSON', Name), label_img)
        count = count + 1
        print('{}/6171 labels are established'.format(count))

流式输出内容被截断，只能显示最后 5000 行内容。
1172/6171 labels are established
1173/6171 labels are established
1174/6171 labels are established
1175/6171 labels are established
1176/6171 labels are established
1177/6171 labels are established
1178/6171 labels are established
1179/6171 labels are established
1180/6171 labels are established
1181/6171 labels are established
1182/6171 labels are established
1183/6171 labels are established
1184/6171 labels are established
1185/6171 labels are established
1186/6171 labels are established
1187/6171 labels are established
1188/6171 labels are established
1189/6171 labels are established
1190/6171 labels are established
1191/6171 labels are established
1192/6171 labels are established
1193/6171 labels are established
1194/6171 labels are established
1195/6171 labels are established
1196/6171 labels are established
1197/6171 labels are established
1198/6171 labels are established
1199/6171 labels are established
1200/6171 labels are established
1201/6171 labels

In [6]:
## Write the list of source and target images
import os

def get_imlist(path, txt):
  data = open(txt, 'w+')
  for f in os.listdir(path):
    if f.endswith('.png'):
      print(f, file = data)
  data.close()

  #return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

Source_list = get_imlist('/content/FDA_Drone_Data/FDA/Label_fromJSON', '/content/Sourcelist.txt')
Target_list = get_imlist('/content/FDA_Drone_Data/FDA/airsim_images/images', '/content/Targetlist.txt')

In [7]:
## Unzip dataset (Airsim with groundtruth)
!unzip /content/drive/MyDrive/blender_airsim_aligned.zip -d /content/FDA_Drone_Data
!rm -rf /content/FDA_Drone_Data/__MACOSX

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_000780.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_000958.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_000794.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_002183.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_004294.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_002815.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_000964.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_000970.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_002801.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_004337.png  
  inflating: /content/FDA_Drone_Data/blender_airsim_aligned/airsim/image_003458.png  
  inflating: /content/FDA_D

In [8]:
## Write labeled images
%cd /content/FDA_Drone_Data/blender_airsim_aligned
!mkdir ./Label_fromJSON

image = './airsim'
info_json =  './simulated_corrosion_multi_class_1.json'

if __name__=='__main__':
    dataset = COCODetection(image, info_json)
    loader = DataLoader(dataset)
    count = 0
    for img, label, name, Cat in loader:
        img = np.uint8(img.squeeze().numpy().transpose(1, 2, 0))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        Name = name[0].replace('image', 'label')
        gt, masks, num_crowds = label
        masks = masks.squeeze(0) ## .squeeze(A): Delete A_th dimension. Now we left batch * H * w, each batch represent a category labeled by 'pixel==1'
        label_img = np.zeros_like(gray)
        Cat_np = []
        for i in Cat:
          Cat_np.append(i.numpy().reshape(-1))
        for m in range(masks.size(0)):
          mask = masks[m].numpy()
          label_img[np.where(mask==1)] = Cat_np[m][0]
        cv2.imwrite(osp.join('/content/FDA_Drone_Data/blender_airsim_aligned/Label_fromJSON', Name), label_img)
        count = count + 1
        print('{}/2767 labels are established'.format(count))

/content/FDA_Drone_Data/blender_airsim_aligned
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
1/2767 labels are established
2/2767 labels are established
3/2767 labels are established
4/2767 labels are established
5/2767 labels are established
6/2767 labels are established
7/2767 labels are established
8/2767 labels are established
9/2767 labels are established
10/2767 labels are established
11/2767 labels are established
12/2767 labels are established
13/2767 labels are established
14/2767 labels are established
15/2767 labels are established
16/2767 labels are established
17/2767 labels are established
18/2767 labels are established
19/2767 labels are established
20/2767 labels are established
21/2767 labels are established
22/2767 labels are established
23/2767 labels are established
24/2767 labels are established
25/2767 labels are established
26/2767 labels are established
27/2767 labels are established
28/2767 labels are established
29/2767 lab

In [9]:
## Write the list of val images
import os

def get_imlist(path, txt):
  data = open(txt, 'w+')
  for f in os.listdir(path):
    if f.endswith('.png'):
      print(f, file = data)
  data.close()

  #return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

val_list = get_imlist('/content/FDA_Drone_Data/blender_airsim_aligned/Label_fromJSON', '/content/Vallist.txt')


In [ ]:
## Train the model
%cd /content/drive/MyDrive/Reverse_DA
!python3 train.py --snapshot-dir='../checkpoints/FDA666' --set='train' --model='DeepLab'  #--restore-from='../checkpoints/FDA/Blender_5000'  --learning-rate=1.0e-4  #--init-weights='../checkpoints/FDA/init_weight/DeepLab_init.pth'


Streaming output truncated to the last 5000 lines.
[it 62050][src seg loss 0.1026][domain loss 1.3863][loss_all 1.4889][loss_val 0.2245][lr 1.0422][4.21s]
[it 62055][src seg loss 0.0269][domain loss 1.3864][loss_all 1.4133][loss_val 6.1652][lr 1.0420][4.20s]
[it 62060][src seg loss 0.0236][domain loss 1.3864][loss_all 1.4100][loss_val 0.5471][lr 1.0417][4.20s]
[it 62065][src seg loss 0.0238][domain loss 1.3863][loss_all 1.4101][loss_val 0.5659][lr 1.0414][4.20s]
[it 62070][src seg loss 0.0175][domain loss 1.3863][loss_all 1.4038][loss_val 0.7791][lr 1.0412][4.20s]
[it 62075][src seg loss 0.0835][domain loss 1.3862][loss_all 1.4697][loss_val 2.5734][lr 1.0409][4.20s]
[it 62080][src seg loss 0.0961][domain loss 1.3863][loss_all 1.4824][loss_val 2.6189][lr 1.0407][4.21s]
[it 62085][src seg loss 0.0859][domain loss 1.3862][loss_all 1.4722][loss_val 1.7833][lr 1.0404][4.20s]
[it 62090][src seg loss 0.0965][domain loss 1.3862][loss_all 1.4827][loss_val 1.1170][lr 1.0401][4.20s]
[it 62095][sr

In [ ]:
## Show the labeled image extracted from json File 
image = '/content/FDA_Drone_Data/blender_airsim_aligned/airsim'
info_json =  '/content/FDA_Drone_Data/blender_airsim_aligned/simulated_corrosion_multi_class_1.json'
if __name__=='__main__':
    dataset = COCODetection(image, info_json)
    loader = DataLoader(dataset)
    for img, label, name, Cat in loader:
        img = np.uint8(img.squeeze().numpy().transpose(1, 2, 0))
        print(name[0], '\n', Cat)
        #for i in Cat:
        #  print((i.numpy())[0])
        gt, masks, num_crowds = label
        masks = masks.squeeze(0) ## .squeeze(A): Delete A_th dimension. Now we left batch * H * w, each batch represent a category labeled by 'pixel==1'
        color = np.array([[0, 0, 255], [0, 255, 0], [255, 0, 0]])
        for m in range(masks.size(0)):
            mask = masks[m].numpy()
            y, x = np.where(mask == 1)
            img[y, x, :] = color[[Cat[m].numpy()][0] - 1]
        cv2_imshow(img)
        cv2.waitKey(500)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA/Blender_59815' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'


/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 42.56
20 / 2767: 47.60
30 / 2767: 44.26
40 / 2767: 42.63
50 / 2767: 44.62
60 / 

In [ ]:
!rm -rf /content/results

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA/Blender_51015' --data-dir-target='/content/FDA_Drone_Data/FDA/images' --data-list-target='/content/Sourcelist.txt' --gt_dir='/content/FDA_Drone_Data/FDA/Label_fromJSON' --devkit_dir='/content/Sourcelist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
2800 processd
2900 processd
3000 processd
3100 processd
3200 processd
3300 processd
3400 processd
3500 processd
3600 processd
3700 processd
3800 processd
3900 processd
4000 processd
4100 processd
4200 processd
4300 processd
4400 processd
4500 processd
4600 processd
4700 processd
4800 processd
4900 processd
5000 processd
5100 processd
5200 processd
5300 processd
5400 processd
5500 processd
5600 processd
5700 processd
5800 processd
5900 processd
6000 processd
6100 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA/Blender_51290' --data-dir-target='/content/FDA_Drone_Data/FDA/images' --data-list-target='/content/Sourcelist.txt' --gt_dir='/content/FDA_Drone_Data/FDA/Label_fromJSON' --devkit_dir='/content/Sourcelist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
2800 processd
2900 processd
3000 processd
3100 processd
3200 processd
3300 processd
3400 processd
3500 processd
3600 processd
3700 processd
3800 processd
3900 processd
4000 processd
4100 processd
4200 processd
4300 processd
4400 processd
4500 processd
4600 processd
4700 processd
4800 processd
4900 processd
5000 processd
5100 processd
5200 processd
5300 processd
5400 processd
5500 processd
5600 processd
5700 processd
5800 processd
5900 processd
6000 processd
6100 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the 

In [10]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70105' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.99
20 / 2767: 28.72
30 / 2767: 29.08
40 / 2767: 27.56
50 / 2767: 29.53
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70130' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.28
20 / 2767: 28.05
30 / 2767: 27.82
40 / 2767: 26.36
50 / 2767: 28.02
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70285' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.16
20 / 2767: 27.86
30 / 2767: 27.74
40 / 2767: 26.11
50 / 2767: 27.68
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70375' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.24
20 / 2767: 28.04
30 / 2767: 28.09
40 / 2767: 26.48
50 / 2767: 28.12
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70475' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.78
20 / 2767: 27.27
30 / 2767: 26.88
40 / 2767: 25.24
50 / 2767: 26.67
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70645' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 24.00
20 / 2767: 29.04
30 / 2767: 29.29
40 / 2767: 28.06
50 / 2767: 30.22
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70725' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.43
20 / 2767: 28.30
30 / 2767: 28.27
40 / 2767: 26.77
50 / 2767: 28.58
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70760' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.93
20 / 2767: 27.56
30 / 2767: 27.35
40 / 2767: 25.77
50 / 2767: 27.38
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70790' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.46
20 / 2767: 28.04
30 / 2767: 28.16
40 / 2767: 26.74
50 / 2767: 28.53
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70840' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 24.07
20 / 2767: 29.27
30 / 2767: 29.34
40 / 2767: 28.17
50 / 2767: 30.37
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_70975' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.08
20 / 2767: 27.77
30 / 2767: 27.66
40 / 2767: 26.15
50 / 2767: 27.79
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71045' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 25.40
20 / 2767: 31.54
30 / 2767: 31.29
40 / 2767: 29.70
50 / 2767: 32.01
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71135' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.75
20 / 2767: 28.78
30 / 2767: 29.11
40 / 2767: 27.61
50 / 2767: 29.84
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71230' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 25.52
20 / 2767: 31.36
30 / 2767: 31.18
40 / 2767: 29.64
50 / 2767: 31.89
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71255' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.99
20 / 2767: 27.67
30 / 2767: 27.48
40 / 2767: 25.92
50 / 2767: 27.54
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71485' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.25
20 / 2767: 28.03
30 / 2767: 28.03
40 / 2767: 26.65
50 / 2767: 28.47
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71615' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.22
20 / 2767: 28.07
30 / 2767: 28.06
40 / 2767: 26.57
50 / 2767: 28.33
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71690' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.57
20 / 2767: 28.73
30 / 2767: 28.90
40 / 2767: 27.60
50 / 2767: 29.78
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71725' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.20
20 / 2767: 27.74
30 / 2767: 27.72
40 / 2767: 26.24
50 / 2767: 27.87
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71895' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.74
20 / 2767: 28.54
30 / 2767: 28.89
40 / 2767: 27.29
50 / 2767: 29.21
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_71995' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 24.07
20 / 2767: 29.16
30 / 2767: 29.54
40 / 2767: 28.12
50 / 2767: 30.35
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72075' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.85
20 / 2767: 27.40
30 / 2767: 27.09
40 / 2767: 25.48
50 / 2767: 27.09
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72150' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.91
20 / 2767: 27.47
30 / 2767: 27.25
40 / 2767: 25.59
50 / 2767: 27.16
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72210' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.73
20 / 2767: 27.44
30 / 2767: 27.12
40 / 2767: 25.53
50 / 2767: 27.11
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72235' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.31
20 / 2767: 28.21
30 / 2767: 28.17
40 / 2767: 26.65
50 / 2767: 28.40
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72730' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.80
20 / 2767: 27.51
30 / 2767: 27.17
40 / 2767: 25.54
50 / 2767: 27.12
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72805' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.32
20 / 2767: 28.13
30 / 2767: 28.09
40 / 2767: 26.62
50 / 2767: 28.30
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_72865' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.76
20 / 2767: 28.83
30 / 2767: 29.04
40 / 2767: 27.59
50 / 2767: 29.79
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73130' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.52
20 / 2767: 28.54
30 / 2767: 28.68
40 / 2767: 27.14
50 / 2767: 29.08
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73220' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.17
20 / 2767: 27.80
30 / 2767: 28.01
40 / 2767: 26.56
50 / 2767: 28.33
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73250' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.27
20 / 2767: 28.02
30 / 2767: 28.12
40 / 2767: 26.60
50 / 2767: 28.29
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73585' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.92
20 / 2767: 27.59
30 / 2767: 27.46
40 / 2767: 25.86
50 / 2767: 27.45
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73855' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.77
20 / 2767: 27.42
30 / 2767: 27.07
40 / 2767: 25.54
50 / 2767: 27.06
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_73970' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.42
20 / 2767: 28.17
30 / 2767: 28.24
40 / 2767: 26.70
50 / 2767: 28.32
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_74015' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.04
20 / 2767: 27.66
30 / 2767: 27.54
40 / 2767: 26.04
50 / 2767: 27.56
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_74145' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.62
20 / 2767: 28.53
30 / 2767: 28.67
40 / 2767: 27.26
50 / 2767: 29.32
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_74265' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.46
20 / 2767: 28.55
30 / 2767: 28.58
40 / 2767: 27.27
50 / 2767: 29.39
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_74595' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.31
20 / 2767: 28.19
30 / 2767: 28.24
40 / 2767: 26.68
50 / 2767: 28.37
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_74785' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 25.84
20 / 2767: 32.28
30 / 2767: 31.80
40 / 2767: 30.03
50 / 2767: 32.16
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75035' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 24.60
20 / 2767: 30.15
30 / 2767: 30.26
40 / 2767: 28.86
50 / 2767: 31.06
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75095' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 26.16
20 / 2767: 32.69
30 / 2767: 32.15
40 / 2767: 30.33
50 / 2767: 32.53
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75105' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.93
20 / 2767: 28.81
30 / 2767: 29.20
40 / 2767: 27.82
50 / 2767: 29.99
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75185' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.93
20 / 2767: 29.14
30 / 2767: 29.52
40 / 2767: 28.19
50 / 2767: 30.38
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75225' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.79
20 / 2767: 28.83
30 / 2767: 29.18
40 / 2767: 27.68
50 / 2767: 29.83
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75625' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.18
20 / 2767: 27.85
30 / 2767: 27.85
40 / 2767: 26.14
50 / 2767: 27.71
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75690' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.81
20 / 2767: 27.46
30 / 2767: 27.10
40 / 2767: 25.56
50 / 2767: 27.08
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75745' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 24.06
20 / 2767: 29.08
30 / 2767: 29.44
40 / 2767: 28.07
50 / 2767: 30.24
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_75825' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.30
20 / 2767: 28.10
30 / 2767: 28.01
40 / 2767: 26.36
50 / 2767: 27.96
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76135' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.41
20 / 2767: 28.47
30 / 2767: 28.50
40 / 2767: 27.06
50 / 2767: 28.96
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76265' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.95
20 / 2767: 27.78
30 / 2767: 27.40
40 / 2767: 25.84
50 / 2767: 27.41
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76295' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.84
20 / 2767: 27.57
30 / 2767: 27.35
40 / 2767: 25.69
50 / 2767: 27.27
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76500' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.88
20 / 2767: 27.45
30 / 2767: 27.19
40 / 2767: 25.56
50 / 2767: 27.18
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76820' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 26.55
20 / 2767: 32.85
30 / 2767: 32.41
40 / 2767: 31.36
50 / 2767: 33.44
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76900' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.20
20 / 2767: 27.99
30 / 2767: 28.05
40 / 2767: 26.46
50 / 2767: 28.08
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_76915' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.61
20 / 2767: 27.13
30 / 2767: 26.69
40 / 2767: 25.03
50 / 2767: 26.28
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_77240' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.88
20 / 2767: 27.77
30 / 2767: 27.43
40 / 2767: 26.03
50 / 2767: 27.77
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_77310' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.40
20 / 2767: 28.31
30 / 2767: 28.39
40 / 2767: 26.87
50 / 2767: 28.73
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_77390' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.91
20 / 2767: 27.56
30 / 2767: 27.35
40 / 2767: 25.72
50 / 2767: 27.30
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_77770' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 21.82
20 / 2767: 26.02
30 / 2767: 24.97
40 / 2767: 23.31
50 / 2767: 23.85
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_78120' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.84
20 / 2767: 27.51
30 / 2767: 27.21
40 / 2767: 25.56
50 / 2767: 27.11
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_78220' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.30
20 / 2767: 28.16
30 / 2767: 28.29
40 / 2767: 26.78
50 / 2767: 28.66
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_78725' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.39
20 / 2767: 28.22
30 / 2767: 28.58
40 / 2767: 27.17
50 / 2767: 29.25
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_78865' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.83
20 / 2767: 27.52
30 / 2767: 27.37
40 / 2767: 25.65
50 / 2767: 27.18
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_78975' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.81
20 / 2767: 28.94
30 / 2767: 29.32
40 / 2767: 28.05
50 / 2767: 30.29
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79345' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 26.17
20 / 2767: 32.51
30 / 2767: 31.95
40 / 2767: 30.30
50 / 2767: 32.41
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79550' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.60
20 / 2767: 27.08
30 / 2767: 26.50
40 / 2767: 24.86
50 / 2767: 26.10
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79590' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.73
20 / 2767: 27.29
30 / 2767: 27.05
40 / 2767: 25.34
50 / 2767: 26.80
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79625' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 22.65
20 / 2767: 27.28
30 / 2767: 26.81
40 / 2767: 25.13
50 / 2767: 26.53
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79700' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.68
20 / 2767: 28.51
30 / 2767: 28.98
40 / 2767: 27.50
50 / 2767: 29.67
60 / 

In [ ]:
%cd /content/drive/MyDrive/Reverse_DA
!python3 Test.py --model='DeepLab' --save='/content/results' --restore-opt1='/content/drive/MyDrive/checkpoints/FDA666/Blender_79745' --data-dir-target='/content/FDA_Drone_Data/blender_airsim_aligned/airsim' --data-list-target='/content/Vallist.txt'
!rm -rf /content/results

/content/drive/MyDrive/Reverse_DA
0 processd
100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
---------Compute IoU------------------
Test.py:61: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(['background', 'Seam', 'Edge', 'Spot'], dtype=np.str)
10 / 2767: 23.35
20 / 2767: 28.25
30 / 2767: 28.29
40 / 2767: 26.79
50 / 2767: 28.59
60 / 